In [71]:
from utils.FEniCSx_solver import FEniCSx_solver
from dolfinx import mesh as msh, fem, default_scalar_type, la
from mpi4py import MPI
import ufl
import numpy as np

import scipy

nx=16
ny=16
comm = MPI.COMM_WORLD
cell_type=msh.CellType.triangle
mesh = msh.create_unit_square(comm=comm, nx=nx, ny=ny, cell_type=cell_type)

Wh = fem.functionspace(mesh, ('P', 1))
uh = fem.Function(Wh)
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
boundary_facets = msh.exterior_facet_indices(mesh.topology)
boundary_dofs = fem.locate_dofs_topological(Wh, mesh.topology.dim-1, boundary_facets)
eps = fem.Constant(mesh, default_scalar_type(1e-8))
b = ufl.as_vector((fem.Constant(mesh, default_scalar_type(1.0)),fem.Constant(mesh, default_scalar_type(0.0))))
c = fem.Constant(mesh, default_scalar_type(0.0))
f = fem.Constant(mesh, default_scalar_type(1.0))
bcs = [fem.dirichletbc(fem.Constant(mesh, default_scalar_type(0.0)), boundary_dofs, Wh)]

pde_data = mesh,Wh,uh,eps,b,None,f,None,bcs



cid_lims = mesh.topology.index_map(2).local_range
marker_ids = np.arange(cid_lims[0], cid_lims[1])

for index in range(cid_lims[0],cid_lims[1]):
    if np.intersect1d(Wh.dofmap.cell_dofs(index), bcs[0].dof_indices()[0]).size > 0:
            marker_ids = marker_ids[marker_ids!=index]

marker = np.ones_like(marker_ids, dtype=np.int32)
cell_tag = msh.meshtags(mesh, mesh.topology.dim, marker_ids, marker)
dx = ufl.Measure("dx", domain=mesh, subdomain_data=cell_tag, subdomain_id=1)

x = ufl.SpatialCoordinate(mesh)
ex_exp = x[0]*(1-ufl.exp(-(1-x[0])/eps))* (1 - (((ufl.exp(-(1-x[1])/eps)  + ufl.exp(-(x[1])/eps)))- ufl.exp(-(1)/eps))/(1-ufl.exp(-1/eps)))

exp = fem.Expression(ex_exp, Wh.element.interpolation_points())

u_ex = fem.Function(Wh)
u_ex.interpolate(exp)

n=ufl.FacetNormal(mesh)
residual = (-eps*ufl.div(ufl.grad(uh)) + ufl.dot(b, ufl.grad(uh)) + c * uh - f)**2 * dx

b_perp = ufl.as_vector((fem.Constant(mesh, default_scalar_type(0.0)),fem.Constant(mesh, default_scalar_type(-1.0))))
cross = abs(ufl.dot(b_perp, ufl.grad(uh)))
crosswind_loss = ufl.conditional(ufl.lt(cross, 1), 1/2*(5*cross**2 - 3*cross**3), ufl.sqrt(cross)) * dx
#loss = (uh-u_ex)**2 * ufl.dx
loss = residual + crosswind_loss
fs = FEniCSx_solver(pde_data=pde_data, loss_form=loss)


norm_b = ufl.sqrt(ufl.dot(b,b))
h = ufl.CellDiameter(domain=mesh) 
alpha = norm_b*h/(2*eps)
Xi = (1/ufl.tanh(alpha)-1/alpha)
tau_K = h/(2*norm_b)*Xi
Th = fem.functionspace(mesh, ('DG', 0))
tau = fem.Function(Th)
tau_exp = fem.Expression(tau_K, Th.element.interpolation_points())
tau.interpolate(tau_exp)
fs.set_weights(tau.x.array)


arr = np.array([])


In [72]:
from scipy.optimize import minimize
    

def callback(intermediate_result):
    fval = intermediate_result.fun
    print(f"J: {fval}")

def eval(weights):
    fs.set_weights(weights=weights)
    return fs.loss()

def eval_grad(weights):
    return fs.grad()

bounds = scipy.optimize.Bounds(lb=np.zeros_like(fs.yh.x.array), ub=10*fs.yh.x.array)
minimize(
    fun=eval,
    x0=fs.yh.x.array,
    jac=eval_grad,
    method='L-BFGS-B',
    tol=1e-9,
    options={"disp": True},
    callback=callback,
    bounds=bounds
)

J: 0.21066114904794309
J: 0.2073403298114191
J: 0.1935721436812248
J: 0.19157791566664475
J: 0.18943558592896081
J: 0.18892566573142056
J: 0.18755284861184207
J: 0.18682496487503075
J: 0.18573035139969915
J: 0.1829463864853213
J: 0.18130954272844585
J: 0.1792640897323132
J: 0.17874926803219376
J: 0.17835637098543483
J: 0.17804105909933532
J: 0.17776447148920474
J: 0.17746661593481705
J: 0.17688532912997196
J: 0.17659240523123537
J: 0.17608223657534938
J: 0.174848247392
J: 0.1735420926431514
J: 0.17181040935564862
J: 0.16909622969931506
J: 0.16755556214100395
J: 0.1600279621009378
J: 0.15351514030643187
J: 0.14497183685690077
J: 0.14309522377701678
J: 0.13889255887133709
J: 0.13666887722963417
J: 0.13404136648494347
J: 0.13118930870360473
J: 0.12816014898356687
J: 0.12471044829953878
J: 0.12230189327304894
J: 0.12110802242737127
J: 0.12023648700504988
J: 0.11904159644189932
J: 0.11855952675792314
J: 0.11772562984058034
J: 0.11679756926358706
J: 0.11586228790461127
J: 0.11497661389607544

  message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
  success: True
   status: 0
      fun: 0.016162583552574425
        x: [ 4.419e-02  4.127e-02 ...  4.419e-01  4.419e-02]
      nit: 3434
      jac: [ 0.000e+00  1.928e-04 ... -2.469e-03  0.000e+00]
     nfev: 3612
     njev: 3612
 hess_inv: <512x512 LbfgsInvHessProduct with dtype=float64>

In [73]:
from utils.plotter import fem_plotter_grid
import pyvista as pv
pv.global_theme.cmap = 'coolwarm'
n = ufl.FacetNormal(mesh)
#residual = (-eps*ufl.div(ufl.grad(fs.uh)) + ufl.dot(b, ufl.grad(fs.uh)) + c * fs.uh - f)**2 * ufl.TestFunction(Wh) * dx
residual = (ufl.dot(b, ufl.grad(fs.uh))-f)**2 * ufl.TestFunction(Wh) * ufl.dx 

#b_perp = ufl.conditional(ufl.eq(ufl.inner(b, b),0), b, -ufl.perp(b)/ufl.sqrt(ufl.inner(b,b)))
cross = ufl.max_value(ufl.dot(b_perp, ufl.grad(fs.uh)), -ufl.dot(b_perp, ufl.grad(fs.uh)))
crosswind_loss = ufl.conditional(ufl.lt(cross, 1), 1/2*(5*cross**2 - 3*cross**3), ufl.sqrt(cross)) * ufl.TestFunction(Wh) * ufl.dx

local_loss = fem.assemble_vector(fem.form(residual)).array


grid = fem_plotter_grid(Wh)
import pyvista as pv

p1, p2 = pv.Plotter(), pv.Plotter()

grid.add_data(fs.uh)
p1.add_mesh(
    grid.grid.warp_by_scalar(), 
    show_edges=True
)


grid.add_data(local_loss)
p2.add_mesh(
    grid.grid,
    show_edges=True
)
p2.camera_position = 'xy'
p1.show()
p2.show()

Widget(value='<iframe src="http://localhost:53654/index.html?ui=P_0x38afb4050_58&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:53654/index.html?ui=P_0x388e707d0_59&reconnect=auto" class="pyvist…

from dolfinx import io
with io.XDMFFile(MPI.COMM_WORLD, "solution_quads_16x16.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(fs.uh)


with io.XDMFFile(MPI.COMM_WORLD, "params_quads_16x16.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(fs.uh)